In [ ]:
import numpy as np
from utils.utils import rle_decode, rle_encode
from scipy import ndimage

In [ ]:
def remove_disconnected_voxels(voxel_grid, max_distance=5, size_threshold=100):
    # 创建一个结构元素，表示一个体素与其相邻体素之间的连接方式。
    # 在这里我们创建一个三维的球形结构，半径等同于max_distance。
    struct = ndimage.generate_binary_structure(3, 1)
    struct = ndimage.iterate_structure(struct, max_distance)

    # 使用结构元素应用连通组件分析。
    labeled_array, num_features = ndimage.label(voxel_grid, structure=struct)

    # 对每个连通组件进行大小统计。
    component_sizes = np.array(ndimage.sum(voxel_grid, labeled_array, range(num_features + 1)))

    # 移除体素数小于size_threshold的小组件。
    # 小组件对应的索引，为True的将被移除。
    remove_idx = component_sizes < size_threshold
    remove_idx[0] = False  # 索引0总是背景，不应该删除。

    # 将小组件置为0以"移除"它们。
    voxel_grid_removed = labeled_array.copy()
    voxel_grid_removed[remove_idx[labeled_array]] = 0

    # 转换回二值图像：所有非0元素都是主要连通组件。
    voxel_grid_cleaned = voxel_grid_removed > 0

    return voxel_grid_cleaned.astype(np.int)

# 示例使用:
# 假设 voxel_model 是一个numpy的三维数组，其中物体的体素为1，空白为0
# 需要去掉与主体无连接的点，即体素数小于某一阈值（例如100）的连通整体。

# Setup (replace this with your actual voxel model):
voxel_model = np.random.randint(2, size=(30, 30, 30))  

# 应用算法：
cleaned_voxel_model = remove_disconnected_voxels(voxel_model, max_distance=5, size_threshold=100)

# cleaned_voxel_model 现在将是清理后的体素模型